In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train= pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')
print('Train Read Successfully')
test= pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')
print('Test Read Successfully')

In [ ]:
train= train.drop('Id',axis=1)

In [ ]:
train.head()

In [ ]:
import datetime
train['Open Date']= train['Open Date'].astype('datetime64[ns]')



In [ ]:
test['Open Date']= test['Open Date'].astype('datetime64[ns]')

In [ ]:
from datetime import datetime
train['Dummy']= datetime.strptime('2015-09-14', '%Y-%m-%d')


In [ ]:
test['Dummy']=datetime.strptime('2015-09-14', "%Y-%m-%d")

In [ ]:
test['Open Date']= test['Dummy']-test['Open Date']

In [ ]:
test.drop('Dummy', axis=1, inplace=True)


In [ ]:
test.drop('Id', axis=1, inplace=True)

In [ ]:
test.head(2)

In [ ]:
train['Open Date']= train['Dummy']-train['Open Date']

In [ ]:
train.drop('Dummy', axis=1, inplace=True)


In [ ]:
train['Open Date']= train['Open Date'].dt.days

In [ ]:
test['Open Date']= test['Open Date'].dt.days

In [ ]:
train.head(2)

In [ ]:
import seaborn as sns
sns.scatterplot(x=train['Open Date'], y=train['revenue'])

In [ ]:
import numpy as np
np.corrcoef(train['Open Date'], train['revenue'])[0][1]

#So there is a very poor correlation between the opening days and revnue

In [ ]:
#Finding outliers for revenue
sns.boxplot(y=train['revenue'])

#Hence, as we can see we have 3 outliers in the revenue data. Their revenue is greater than 1.25. So we remove them.

In [ ]:
train[train['revenue']>12500000]

In [ ]:
train= train.drop([16,75,99])  #Here we drop the values of the row by index specified.

In [ ]:
train[train['revenue']>12500000]

In [ ]:
#Niw again drawing the boxplot
sns.boxplot(train['revenue'])

In [ ]:
train.head()

In [ ]:
train['City'].value_counts()

In [ ]:
#WE cannot encode 37 values at one toime. So we retain those having 4 or more resturants and rest are treated as others
pd_city= train['City'].reset_index()
pd_city['City'].value_counts()

In [ ]:
pd_city['City'].value_counts().reset_index()[:7]['index'].to_list()

In [ ]:
city_name= ['İstanbul', 'Ankara', 'İzmir', 'Samsun', 'Bursa', 'Sakarya', 'Antalya']

In [ ]:
pd_city.head()

In [ ]:
pd_city['City']

In [ ]:
city_dummy_df= pd.get_dummies(train[['City']] ,prefix=['City'])
city_dummy_df['City_Other']=0
city_dummy_df

In [ ]:
test_dummy_city= pd.get_dummies(test[['City']], prefix='City')
test_dummy_city['City_Other']=0
test_dummy_city

In [ ]:
for index, rows in city_dummy_df.iterrows():  #This will iterate over all the rows of one particluar index
    if (
        rows['City_Istanbul']==0 and
        rows['City_Ankara']==0 and 
        rows['City_İzmir']==0 and 
        rows['City_Samsun']==0 and
        rows['City_Bursa']==0 and 
        rows['City_Sakarya']==0 and 
        rows['City_Antalya']==0
    ):
        city_dummy_df['City_Other'][index]=1
        
        
       
city_dummy_df[city_dummy_df['City_Other']==1].shape[0]

In [ ]:
for index, rows in test_dummy_city.iterrows():
    if (rows['City_İstanbul']==0 and
        rows['City_Ankara']==0 and
        rows['City_İzmir']==0 and 
        rows['City_Samsun']==0 and
        rows['City_Bursa']==0 and 
        rows['City_Sakarya']==0 and 
        rows['City_Antalya']==0
       ):
        test_dummy_city['City_Other'][index]=1
        


In [ ]:
test_dummy_city= test_dummy_city[["City_İstanbul", "City_Ankara", "City_İzmir", "City_Bursa", "City_Samsun", "City_Antalya", "City_Sakarya", "City_Other"]]

In [ ]:
test= pd.merge(test, test_dummy_city, right_index=True, left_index=True)

In [ ]:
city_dummy_df= city_dummy_df[['City_İstanbul',
 'City_Ankara',
 'City_İzmir',
 'City_Samsun',
 'City_Bursa',
 'City_Sakarya',
 'City_Antalya', 'City_Other']]

In [ ]:
city_name
l1=[]
for x in city_name:
    l1.append('City_'+x)
    
l1

In [ ]:
train1= train.copy()
train1

In [ ]:
train= pd.merge(train, city_dummy_df, left_index=True, right_index=True)

In [ ]:
train.head()

In [ ]:
#Lets take a look at city group field 
train['City Group'].value_counts().plot(kind='bar')
#AS there are only 2 values, we hot encode them

In [ ]:
city_df= pd.get_dummies(train[['City Group']], prefix='Group')
city_df

In [ ]:
test_city= pd.get_dummies(test[['City Group']], prefix='Group')
test_city

test= pd.merge(test, test_city, right_index=True, left_index=True)

In [ ]:
train= pd.merge(train, city_df, left_index=True, right_index=True)
train.head(2)

In [ ]:
train['Type'].value_counts()

In [ ]:
test[test['Type']=='DT'].shape
test['Type'].value_counts()

In [ ]:
rev_avg_df = train[["Type", "revenue"]].groupby("Type").mean()
type_freq_df = train[["Type", "revenue"]].groupby("Type").count()
rev_info_by_type_df = pd.merge(type_freq_df, rev_avg_df, on = "Type")
rev_info_by_type_df

In [ ]:
average_revenue= train[['Type', 'revenue']].groupby('Type').mean()
average_revenue

count_type= train[['Type', 'revenue']].groupby('Type').count()
count_type

pd.merge(average_revenue, count_type, left_index=True, right_index=True).sort_values(by='revenue_x', 
                            ascending=False).rename(columns={'revenue_x':'Average_revenue', 'revenue_y':'Count'})

#As DT average revenue is more close to IL, so we merge it with that for better accuracy.



In [ ]:
train[train['Type']=='DT']

In [ ]:
for index, rows in train.iterrows():
    if rows['Type']=='DT':
        train['Type'][index]='IL'

In [ ]:
#For test data we convert DT to IL as in train and MB to FC the most populat type
for index, rows in test.iterrows():
    if rows['Type']=='DT':
        test['Type'][index]=='IL'
    elif rows['Type']=='MB':
        test['Type'][index]=='FC'

In [ ]:
type_test= pd.get_dummies(test[['Type']])
test_type=type_test.drop(['Type_DT', 'Type_MB'], axis=1)

In [ ]:
test= pd.merge(test, test_type, right_index=True, left_index=True)

In [ ]:
train['Type'].value_counts()

In [ ]:
train_type= pd.get_dummies(train[['Type']])
train= pd.merge(train, train_type ,right_index=True, left_index=True)

In [ ]:
train.shape

In [ ]:
train.head(2)

In [ ]:
train

In [ ]:
import numpy as np
numerical_df= train.select_dtypes(np.number)

In [ ]:
numerical_df.corr()
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,10))
sns.heatmap(numerical_df.corr() ,robust=True)

In [ ]:
from sklearn import preprocessing
min_max_scaler= preprocessing.MinMaxScaler()
min_max_scaler

plt.figure(figsize=(20,10))
sns.heatmap(pd.DataFrame(min_max_scaler.fit_transform(numerical_df)).corr(), robust=True)

In [ ]:
abs(train.corr())['revenue'].sort_values(ascending=False).head(10)

#Out of all P's P2 and P28 have the highest correlation with the revenue.

In [ ]:
features_index= [2,28]

l=[]
for i in range(1,38):
    if i not in features_index:
        l.append('P'+ str(i))
        
        
train= train.drop(l, axis=1)

In [ ]:
test= test.drop(l, axis=1)

In [ ]:
from sklearn import linear_model
reg= linear_model.LinearRegression()

In [ ]:
y_train= train[['revenue']]
y_train

x_train= train.drop(rm_columns, axis=1)

In [ ]:
x_train.head()

In [ ]:
train.columns

In [ ]:
rm_columns= ['City', 'City Group', 'Type', 'revenue']
rm_columns

In [ ]:
reg.fit(x_train, y_train)

In [ ]:
x_test= test.drop(['City', 'City Group', 'Type'], axis=1)

In [ ]:
x_test.head()

In [ ]:
x_test.shape
x_train.shape


In [ ]:
y_pred= reg.predict(x_test)

In [ ]:
y_pred

In [ ]:
test['Prediction']= y_pred
submit= test['Prediction'].reset_index().rename(columns={'index':'Id'})
submit

In [ ]:
submit.to_csv('Submission_csv', index=False)

In [ ]:
from sklearn.metrics import r2_score
X= x_train
y=y_train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=22, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf= RandomForestRegressor(n_estimators=200, max_depth=5, min_samples_leaf=100, n_jobs=4, random_state=22)
rf.fit(X_train, y_train)

y_predict= rf.predict(X_test)
y_predict

r2_score(y_test, y_predict)

#Very very low

In [ ]:
from datetime import datetime
datetime.strptime(train.loc[1,'Open Date'], '%m/%d/%Y').day

In [ ]:
for i in range(train.shape[0]):
    train.loc[i, 'Month'] =datetime.strptime(train.loc[i, 'Open Date'], '%m/%d/%Y').month
    train.loc[i, 'Year']= datetime.strptime(train.loc[i, 'Open Date'], '%m/%d/%Y').year

In [ ]:
train['Month']=0
train['Year']=0

In [ ]:
train.head(2)

In [ ]:
month_df= train['Month'].value_counts().reset_index().sort_values(by='index',)
import seaborn as sns
month_df
sns.barplot(x='index', y='Month', data= month_df, palette='winter')

In [ ]:
yeardf= train['Year'].value_counts().reset_index().sort_values(by='index')
yeardf.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))

ax= sns.barplot(x='index', y='Year', data=yeardf)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45, )

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y=p.get_xy()
    ax.annotate("{:.2f}".format(p.get_height()), (x+width/2, y+height+0.5), ha='center')
    
#Hence ma x resturants opened in 2011

In [ ]:
revenue_month= train.groupby('Month')['revenue'].mean().reset_index()
revenue_month

In [ ]:
sns.pointplot(x=revenue_month['Month'], y=revenue_month['revenue'])
#Revenue is highest in the month of Jan


In [ ]:
revenue_year= train.groupby('Year')['revenue'].mean().reset_index()
revenue_year

#Hence max revenue is generated in yer 1999 to 2000

In [ ]:
ax= sns.pointplot(revenue_year['Year'], revenue_year['revenue'])
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)